In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import time
import re 
import random
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import text 
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.io import read_image
from pathlib import Path
from PIL import Image
import numpy as np


from torch.nn.functional import mse_loss as mse

from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as tdata
import torch
import pandas as pd
import numpy as np
import os
import re 
import random
import glob
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.io import read_image

import torch.nn as nn
import torch.nn.functional as F

import pickle # extra
from datetime import datetime
import torchvision.models as models
from kornia.losses import SSIMLoss, PSNRLoss
from kornia.metrics import psnr, ssim
from mpl_toolkits.axes_grid1 import make_axes_locatable

import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
torch.set_printoptions(linewidth=120)

from torchsummary import summary

import pickle
import json
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

torch.manual_seed(1996)
torch.cuda.manual_seed(1996)
np.random.seed(1996)
random.seed(1996)

# Load the required models

In [ ]:
class costumeDataset(Dataset):
    """
    data_path: path of all the images as a list, each element in the list is path of an image
    label_map: dataframe of corresponding labels that is loaded: maps label to image-id
    class_map: a dictionary of classes: maps labels to encoded labels
    patient_map: a dataframe that maps patient_id to frequency of each label among the images of that patient
    label_col: the column in label file where the labels are saved: here is diagnosis
    transform: the type of transform. As default it is none, meaning that the images will only be transformed to tensors
    it can also have augmentations
    noisy_transform: the type of noise implemented in the image. default: none, no noisy data
    val_split: val_train ratio. default: None, no validation split.
    
    Return Dataset class representing our data set
    """
    def __init__(self, 
                 data_dir, 
                 label_map, 
                 class_map, 
                 #patient_map,
                 label_col, 
                 augment = None, 
                 noisy_transform= None, 
                 image_size = (224,224)
                ):
        
        self.data_dir = data_dir
        self.label_map = label_map #loaded it in the loader once
        self.augment = augment
        self.noisy_transform = noisy_transform
        self.class_map = class_map
        self.label_col = label_col
        self.image_size = image_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Resize(self.image_size),
            transforms.ToTensor()
        ])
        
    def __len__(self): 
        '''returns size of the dataset'''
        return len(self.label_map)

    def __getitem__(self, index): 
        'Generates one sample of data'  
        img_path = self.data_dir+"/"+self.label_map.loc[index, self.label_col]
        img = Image.open(img_path)
        
        label = self.class_map[self.label_map.loc[index, "class_label"]]
  
        if self.transform: 
            image = self.transform(img)
        if self.augment: 
            augment_image = self.augment(image)
            if self.noisy_transform: 
                noisy_image = self.noisy_transform(augment_image)
                return augment_image, noisy_image, label

        else: 
            if self.noisy_transform: 
                noisy_image = self.noisy_transform(image)
                return image, noisy_image, label
            
                
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__() 
        # Encoder
        self.conv1 = nn.Conv2d(3, 32, 4, stride=2, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 4, stride=2, padding=1) 
    
        # Decoder       
        self.deconv4 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(128*2, 64, 4, stride=2, padding=1)
       
        self.deconv2 = nn.ConvTranspose2d(64*2, 32, 4, stride=2, padding=1)     
        self.deconv1 = nn.ConvTranspose2d(32*2, 3, 4, stride=2, padding=1)

        self.dropout = nn.Dropout(p=0.50)
        self.act_fn = nn.LeakyReLU(negative_slope=0.2)
        self.out_fn = nn.Sigmoid()
        
    def forward(self, x):       
     
        z1 = self.conv1(x)       
        z1 = self.act_fn(z1)
        z1 = self.dropout(z1)  
        
        z2 = self.conv2(z1)        
        z2 = self.act_fn(z2)
        z2 = self.dropout(z2)
        
        z3 = self.conv3(z2)     
        z3 = self.act_fn(z3)
        z3 = self.dropout(z3)

        z4 = self.conv4(z3)        
        z = self.act_fn(z4) 
        # Decoder
        x_hat = self.deconv4(z)
        x_hat = self.act_fn(x_hat)
        x_hat = torch.cat((x_hat,z3),1)  
        x_hat = self.deconv3(x_hat)
        x_hat = self.act_fn(x_hat)
        
        x_hat = torch.cat((x_hat,z2),1)     
        x_hat = self.deconv2(x_hat)
        x_hat = self.act_fn(x_hat)
        x_hat = torch.cat((x_hat,z1),1)   
        x_hat = self.deconv1(x_hat)
        x_hat = self.out_fn(x_hat)
        return {'z': z, 'x_hat': x_hat}

    
"_______________set experiment__________________"
def set_experiment(model, params):
    # This function initializes all the modules needed to train the model given a set of parameters.
    # Transform
    composed = {
        'augment': transforms.Compose([
            transforms.ToPILImage(),
            #transforms.Resize(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomAffine(degrees=(10), scale=(0.8,1.2)),
            transforms.ToTensor()
            ]),
        
        'noisy': transforms.Compose([
            transforms.ToPILImage(),            
            transforms.RandomApply([transforms.ColorJitter(brightness=[params["brightness"], params["brightness"]])],p=params["p"]),
            transforms.ToTensor()
            ])
    }
    data_dir = "../imagine/images"
    clinic_dir = "../imagine/clinic"
    train_path = "../imagine/train.pickle"
    test_path = '../imagine/test.pickle'
    valid_path = '../imagine/valid.pickle'
    data_report_dir = "../imagine_report/report"
    added_path = "../imagine_report/added.csv"
    light_path = "../imagine_report/light.csv"
    dark_path = "../imagine_report/dark.csv"
    natural_path = "../imagine_report/natural.csv"
    added_appendix_path = "../imagine_report/added_appendix.csv"
    added_report_path = "../imagine_report/added_report.csv"
    clinic_path = "../imagine/clinic.csv"
    
    class_map = {"psoriasis":0}
    label_col = "image_id"

    with open(train_path, 'rb') as f:
        x = pickle.load(f)

    train_file = x.loc[x.class_label.isin(["psoriasis"]), :]
    train_file.reset_index(inplace=True)
    train_file = train_file.loc[:, ~train_file.columns.str.contains('^index')]
  

    with open(test_path, 'rb') as f:
        x = pickle.load(f)
  
    test_file = x.loc[x.class_label.isin(["psoriasis"]), :]
    test_file.reset_index(inplace=True)
    test_file = test_file.loc[:, ~test_file.columns.str.contains('^index')]
    

    with open(valid_path, 'rb') as f:
        x = pickle.load(f)
   
    valid_file = x.loc[x.class_label.isin(["psoriasis"]), :]
    valid_file.reset_index(inplace=True)
    valid_file = valid_file.loc[:, ~valid_file.columns.str.contains('^index')]
    
    added_file, light_file, dark_file, natural_file,added_appendix_file, added_report_file, clinic_file = pd.read_csv(added_path, sep=";"), pd.read_csv(light_path, sep=";"), pd.read_csv(dark_path, sep=";"), pd.read_csv(natural_path, sep=";"), pd.read_csv(added_appendix_path, sep=";"), pd.read_csv(added_report_path, sep=";"),pd.read_csv(clinic_path, sep=";")
    added_file = added_file.loc[:, ~added_file.columns.str.contains('Unnamed')]
    light_file = light_file.loc[:, ~light_file.columns.str.contains('Unnamed')]
    dark_file = dark_file.loc[:, ~dark_file.columns.str.contains('Unnamed')]
    natural_file = natural_file.loc[:, ~natural_file.columns.str.contains('Unnamed')]
    added_appendix_file = added_appendix_file.loc[:, ~added_appendix_file.columns.str.contains('Unnamed')]
    added_report_file = added_report_file.loc[:, ~added_report_file.columns.str.contains('Unnamed')]
    clinic_file = clinic_file.loc[:, ~clinic_file.columns.str.contains('Unnamed')]
    # load the dataset
    train_dataset = costumeDataset(data_dir = data_dir, 
             label_map = train_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = composed["augment"], 
             noisy_transform= composed["noisy"],
             image_size = params['image_size'])
    
    test_dataset = costumeDataset(data_dir = data_dir, 
             label_map = test_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])

    valid_dataset = costumeDataset(data_dir = data_dir, 
             label_map = valid_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size']) 
    # for visualising in the report
    added_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = added_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    light_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = light_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    dark_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = dark_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    natural_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = natural_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    
    added_appendix_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = added_appendix_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    added_report_dataset = costumeDataset(data_dir = data_report_dir, 
             label_map = added_report_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size'])
    clinic_dataset = costumeDataset(data_dir = clinic_dir, 
             label_map = clinic_file, 
             class_map = class_map, 
             label_col = label_col, 
             augment = None, 
             noisy_transform= composed["noisy"], 
             image_size = params['image_size']
    

    #set data iterators
    train_loader = DataLoader(
        train_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    
    test_loader =  DataLoader(
        test_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    

    # for visualising
    added_loader = DataLoader(
        added_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    
    light_loader =  DataLoader(
        light_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )

    dark_loader =  DataLoader(
        dark_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    natural_loader =  DataLoader(
        natural_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    added_appendix_loader =  DataLoader(
        added_appendix_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    added_report_loader =  DataLoader(
        added_report_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    
    clinic_loader =  DataLoader(
        clinic_dataset, batch_size=params['batch_size'], shuffle=False, 
        num_workers=4, pin_memory=False
    )
    
    # Get experiment modules
    experiment_modules ={
    'augment': composed["augment"],
    'noisy': composed["noisy"],
    'train_loader': train_loader, 
    'valid_loader': valid_loader,
    'test_loader': test_loader,
    'added_loader': added_loader,
    'light_loader': light_loader,
    'dark_loader': dark_loader,
    'natural_loader':natural_loader,
    'added_appendix_loader': added_appendix_loader,
    'added_report_loader': added_report_loader,
    'clinic_loader': clinic_loader}
    return experiment_modules
 

In [ ]:
net = Autoencoder()
net.to("cuda")
summary(net, (3,224,224))

# Loading The Benchmark and Classifier Results

In [ ]:
with open('MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_GAMMA0.05_P0.6_BRIGHTNESS0.8.pickle', 'rb') as f:
    x = pickle.load(f)
data_items = x. items()
data_list = list(data_items)
df1 = pd.DataFrame(data_list)
print(df1)


# Get Learning Curves

In [ ]:
# plot learning curves
def get_learningCurve(df, b):
    epoch = df.iloc[0,1]
    train_loss = df.iloc[5,1] #ssim train
    test_loss = df.iloc[14,1] # ssim valid
    psnr_train = df.iloc[7,1]
    psnr_test = df.iloc[15,1]
    
    fig, axs = plt.subplots(1, 2,figsize=(15,10))
    axs[0].plot(epoch,train_loss, label='Train')
    axs[0].plot(epoch,test_loss, label='Test')
    axs[0].set_title('SSIM Loss', fontsize=16, fontweight="bold")
    axs[0].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[0].set_ylabel('SSIM Loss', fontsize=16, fontweight="bold")
    axs[0].legend(loc='best')

    axs[1].plot(epoch,psnr_train, label='Train')
    axs[1].plot(epoch,psnr_test, label='Test')
    axs[1].set_title('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[1].set_ylabel('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].legend(loc='best')
    fig.suptitle(f'Learning Curve on Imagine Dataset With Noise Level {b}', fontsize=20, fontweight="bold")
    fig.tight_layout()
    plt.savefig(f"IMAGINE_RESULTS/LearningCurve_BENCHMARK_IMAGINE_LARGEEPOCH_{b}.png")     
    plt.show()
    return 



In [ ]:
# plot learning curves for exp 1 of classifier
def get_learningCurve_class(df, gamma):
    epoch = df.iloc[0,1]
    train_loss = df.iloc[1,1] #total loss train
    valid_loss = df.iloc[8,1] #total loss valid
    psnr_train = df.iloc[7,1]
    psnr_valid = df.iloc[15,1]
    acc_train = df.iloc[6,1]
    acc_valid = df.iloc[9,1]
    
    fig, axs = plt.subplots(1, 3,figsize=(15,10))
    axs[0].plot(epoch,train_loss, label='Train')
    axs[0].plot(epoch,valid_loss, label='Validation')
    axs[0].set_title('Total Loss', fontsize=16, fontweight="bold")
    axs[0].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[0].set_ylabel('Total Loss', fontsize=16, fontweight="bold")
    axs[0].legend(loc='best')

    axs[1].plot(epoch,psnr_train, label='Train')
    axs[1].plot(epoch,psnr_valid, label='Validation')
    axs[1].set_title('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[1].set_ylabel('PSNR Score', fontsize=16, fontweight="bold")
    axs[1].legend(loc='best')
    
    axs[2].plot(epoch,acc_train, label='Train')
    axs[2].plot(epoch,acc_valid, label='Validation')
    axs[2].set_title('Accuracy Score', fontsize=16, fontweight="bold")
    axs[2].set_xlabel('Epoch', fontsize=16, fontweight="bold")
    axs[2].set_ylabel('Accuracy Score', fontsize=16, fontweight="bold")
    axs[2].legend(loc='best')
    
    
    fig.suptitle(f'Learning Curve on Imagine Dataset With Noise Level 0.4 and Classifier Weight of {gamma}', fontsize=20, fontweight="bold")
    fig.tight_layout()
    plt.savefig(f"LearningCurve_CLASSIFIER_ISIC_0.4_WEIGHT{gamma}.png")     
    plt.show()
    return 



In [ ]:
get_learningCurve(df5, 0.8)

# Get psnr and ssim of the last epoch in benchmark

In [ ]:
def get_ssim_psnr_noisy_model(model_file,b, params):
    model = torch.load(model_file)
    model.cuda()
    experiment_modules = set_experiment(model, params)

    
    #setup metrics
    rssimnx = 0.0
    rpsnrnx = 0.0
    rssimxh = 0.0
    rpsnrxh = 0.0
    
    model.eval()
    with torch.inference_mode():
        
        for x, noisy_x, y in experiment_modules["valid_loader"]:
            x, noisy_x, y= x.cuda(), noisy_x.cuda(), y.cuda()
            
            outputs = model(noisy_x) 
            x_hat = outputs['x_hat']

            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            print("msenoisyxtmp", msenoisyxtmp.size())
            print("msexhatxtmp", msexhatxtmp.size())
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            print("msenoisyx",msenoisyx.size())
            print("msexhatx",msexhatx.size())
            print("msenoisyx",msenoisyx.size())
            print("msexhatx",msexhatx.size())
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            print("psnrnoisyx",psnrnoisyx.size())
            print("psnrxxhat",psnrxxhat.size())
            print("psnrnoisyx",psnrnoisyx)
            print("psnrxxhat",psnrxxhat)
            
            
            
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            rssimnx += ssimnoisyx.sum(0)
            rssimxh += ssimxxhat.sum(0)
            
            rpsnrxh += psnrxxhat.sum(0)
            rpsnrnx += psnrnoisyx.sum(0)
            #visualise
            
            plt.imshow(np.transpose(x[2].to("cpu"), (1, 2, 0)))
            plt.show()
            plt.imshow(np.transpose(noisy_x[2].to("cpu"), (1, 2, 0)))
            plt.show()
            plt.imshow(np.transpose(x_hat[2].to("cpu"), (1, 2, 0)))
            plt.show()
            print("ssim of this image", ssimnoisyx[2].item(), ssimxxhat[2].item())
            print("psnr of this image", psnrnoisyx[2].item(), psnrxxhat[2].item())
                 
    n = 1893
    meanssimnx = rssimnx/n
    meanssimxh = rssimxh/n
    
    meanpsnrnx = rpsnrnx/n
    meanpsnrxh = rpsnrxh/n
    return meanssimnx, meanssimxh, meanpsnrnx, meanpsnrxh


In [ ]:
# save the psnr ssim table
def save_psnr_ssim_table(model_b):
    
    shared_params = {
     'batch_size': 30,
     'random_seed': 1996,
     'image_size': (224,224),
     'n_classes': 1
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}epoch_180"

    dic = {"noise_level":[], "mean_ssim_noisy_x":[], "mean_ssim_xhat_x":[] ,"mean_psnr_noisy_x":[], "mean_psnr_xhat_x":[]}
    b_list = np.arange(round(1.0-model_b,1), round(1.0+model_b+0.2,1), 0.2)
    print("=======================================")
    print("model_file", model_file, "model_b", model_b, "Noise_Level_Range", b_list)
    for b in b_list:
        params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": round(b,1) # is noise level
            }
        params = {**shared_params, **params} 
        print("noise_level", round(b,1), params["brightness"])
        meanssimnx, meanssimxh, meanpsnrnx, meanpsnrxh = get_ssim_psnr_noisy_model(model_file, round(b,1), params)
        dic["noise_level"].append(round(b,1)) 
        dic["mean_ssim_noisy_x"].append(meanssimnx.item()) 
        dic["mean_ssim_xhat_x"].append(meanssimxh.item()) 
        dic["mean_psnr_noisy_x"].append(meanpsnrnx.item()) 
        dic["mean_psnr_xhat_x"].append(meanpsnrxh.item())
        with open(f'IMAGINE_RESULTS/MODEL{model_b}_SSIM_PSNR_TABLE.pickle','wb') as handle: 
            pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return dic



In [ ]:
model_b = 0.8
dic = save_psnr_ssim_table(model_b)

# Visualisation of Reconstruction In Appendix

In [ ]:
def visualise_report_appendix(model_file, b_list, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(len(b_list)*2+1,shared_params["batch_size"] ,figsize=(50,50), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    added_noise_labels_appendix = ["A","B","C","D","E","F","H","I","J","K", "L"]
    model.cuda()
    
    for b in range(len(b_list)):
        params = { 
            "gamma": 0.01,
            "brightness": round(b_list[b],1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_appendix_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(noisy_x) 
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                #plot images
                axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #GT
                axs[b*2+1,i].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
                axs[b*2+2,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
                axs[0,i].set_xticklabels([])
                axs[0,i].set_yticklabels([])
                axs[b*2+1,i].set_xticklabels([])
                axs[b*2+1,i].set_yticklabels([])
                axs[b*2+2,i].set_xticklabels([])
                axs[b*2+2,i].set_yticklabels([])
                axs[0,i].set_aspect('equal')
                axs[b*2+1,i].set_aspect('equal')
                axs[b*2+2,i].set_aspect('equal')
                # add ssim and psnr of each
                axs[16,i].set_xlabel(added_noise_labels_appendix[i], fontweight = "bold", fontsize = 25)
                if i == 0:
                    axs[0,i].set_ylabel("GT", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=20)
                    axs[b*2+1,i].set_ylabel(f"{b_list[b]}", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=60)
                    axs[b*2+2,i].set_ylabel(f"SDAE {b_list[b]}", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=80)    
                
                # add ssim and psnr of each
                axs[b*2+1,i].text(x=10,y=10,s=f'{round(ssimnoisyx[i].item(),2)}\n{round(psnrnoisyx[i].item(),2)}',horizontalalignment='left',verticalalignment='top', fontsize=15, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))                    
                axs[b*2+2,i].text(x=10,y=10,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='left',verticalalignment='top', fontsize=15, fontweight="demi", bbox=dict(facecolor="white", alpha=0.5))
               
    fig.suptitle('De-noising Test Images With Different Levels of Added Noise', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    name = f"IMAGINE_RESULTS/VISUALISE_IMAGINE_0.8_LARGEEPOCH_NOISERANGE{b_list}.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name)     
    plt.show()
    return ssim_psnr_values

def save_added_appendix(model_b):
    
    shared_params = {
     'batch_size': 11,
     'random_seed': 1996,
     'image_size': (224,224),
     'n_classes': 1,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}epoch_180"

    b_list = [0.2,0.4,0.6,0.8,1.2,1.4,1.6, 1.8]
    
    ssim_psnr_values = visualise_report_appendix(model_file, b_list, shared_params)
    return ssim_psnr_values

ssim_psnr_values = save_added_appendix(0.8)


# Images Visualised in Report

In [ ]:
def visualise_report(model_file, b_list, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(len(b_list)*2+1,shared_params["batch_size"] ,figsize=(20,30), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    added_noise_labels_appendix = ["A","B","C","D"]
    model.cuda()
    
    for b in range(len(b_list)):
        params = { 
            "gamma": 0.01,
            "brightness": round(b_list[b],1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_report_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(noisy_x) 
            #y_hat = outputs['y_hat'].cuda()
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                
                # add ssim and psnr of each
                #plot images
                axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #GT
                axs[b*2+1,i].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
                axs[b*2+2,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
                axs[0,i].set_xticklabels([])
                axs[0,i].set_yticklabels([])
                axs[b*2+1,i].set_xticklabels([])
                axs[b*2+1,i].set_yticklabels([])
                axs[b*2+2,i].set_xticklabels([])
                axs[b*2+2,i].set_yticklabels([])
                axs[0,i].set_aspect('equal')
                axs[b*2+1,i].set_aspect('equal')
                axs[b*2+2,i].set_aspect('equal')
                # add ssim and psnr of each
                axs[8,i].set_xlabel(added_noise_labels_appendix[i], fontweight = "bold", fontsize = 15)
                if i == 0:
                    axs[0,i].set_ylabel("GT", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=20)
                    axs[b*2+1,i].set_ylabel(f"{b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=30)
                    axs[b*2+2,i].set_ylabel(f"SDAE {b_list[b]}", fontweight = "bold", fontsize = 15, rotation = 0, labelpad=40)    
                
                # add ssim and psnr of each

                axs[b*2+1,i].text(x=10,y=10,s=f'{round(ssimnoisyx[i].item(),2)}\n{round(psnrnoisyx[i].item(),2)}',horizontalalignment='left',verticalalignment='top', fontsize=15, fontweight="demi", bbox=dict(facecolor='white', alpha=0.5))                    
                axs[b*2+2,i].text(x=10,y=10,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='left',verticalalignment='top', fontsize=15, fontweight="demi", bbox=dict(facecolor="white", alpha=0.5)) 
          

    fig.suptitle('De-noising Clinical Images With Different Levels of Added Noise', fontsize=25, fontweight="bold")
    fig.subplots_adjust(wspace=0.01, hspace=0.01, top=0.95, bottom=0.05)
    name = f"IMAGINE_RESULTS/VISUALISE_IMAGINE_0.8_LARGEEPOCH_NOISERANGE{b_list}.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name)     
    plt.show()
    return ssim_psnr_values

def save_added_report(model_b):
    
    shared_params = {
     'batch_size': 4,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 1,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS{model_b}epoch_180"

    b_list = [0.2,0.6,1.4, 1.8]
    ssim_psnr_values = visualise_report(model_file, b_list, shared_params)
    return ssim_psnr_values

ssim_psnr_values = save_added_report(0.8)


# model in different epochs

In [ ]:
def visualise_images_epochs(epoch_list, b, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(shared_params["batch_size"], len(epoch_list)+2 ,figsize=(40,40), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    
    for m in range(len(epoch_list)):
        model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_{epoch_list[m]}"
        model = torch.load(model_file)
        model.cuda()
        params = { 
            "gamma": 0.01,
            "brightness": round(b,1) # is noise level
            }
        params = {**shared_params, **params}
        experiment_modules = set_experiment(model, params)
        model.eval()
        with torch.inference_mode():
            x, noisy_x, y = next(iter(experiment_modules["added_report_loader"]))
            x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
            outputs = model(noisy_x) 
            x_hat = outputs['x_hat']
            # qualitative scores
            ssimnoisyxtmp = ssim(x,noisy_x, 5) #same dimension as x and noisy_x BxCxHxW
            ssimnoisyx = ssimnoisyxtmp.mean(-1).mean(-1).mean(-1) # size = B
            ssimxxhattmp = ssim(x,x_hat, 5) #same dimension as x and noisy_x BxCxHxW
            ssimxxhat = ssimxxhattmp.mean(-1).mean(-1).mean(-1) # size = B
            
            msenoisyxtmp = mse(x,noisy_x, reduction = "none") # BxCxHxW
            msexhatxtmp = mse(x,x_hat, reduction = "none")# BxCxHxW
            msenoisyx = msenoisyxtmp.mean(-1).mean(-1).mean(-1) #B
            msexhatx = msexhatxtmp.mean(-1).mean(-1).mean(-1) #B
            psnrnoisyx = 10.0 * torch.log10(1.0 ** 2 / msenoisyx) #B
            psnrxxhat = 10.0 * torch.log10(1.0 ** 2 / msexhatx) #B
            
            
            ssim_psnr_values["ssim_noisy_x"].append(ssimnoisyx)
            ssim_psnr_values["ssim_xhat_x"].append(ssimxxhat)
            ssim_psnr_values["psnr_noisy_x"].append(psnrnoisyx)
            ssim_psnr_values["psnr_xhat_x"].append(psnrxxhat)
            
            for i in range(params["batch_size"]):
                #plot images
                axs[i,0].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #GT
                axs[i,1].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
                axs[i,m+2].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
                axs[i,0].set_xticklabels([])
                axs[i,0].set_yticklabels([])
                axs[i,1].set_xticklabels([])
                axs[i,1].set_yticklabels([])
                axs[i,m+2].set_xticklabels([])
                axs[i,m+2].set_yticklabels([])
                axs[i,0].set_aspect('equal')
                axs[i,1].set_aspect('equal')
                axs[i,m+2].set_aspect('equal')
                if i == params["batch_size"]-1:
                    axs[i,0].set_xlabel("GT", fontweight = "bold", fontsize = 15, rotation=45)
                    axs[i,1].set_xlabel(f"Noisy(0.6)", fontweight = "bold", fontsize = 15, rotation = 45)
                    axs[i,m+2].set_xlabel(f"De-noised in Epoch({epoch_list[m]})", fontweight = "bold", fontsize = 15, rotation = 45)
                
                # add ssim and psnr of each
                axs[i,m+2].text(x=right,y=bottom,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='left',verticalalignment='top', bbox=dict(facecolor='white', alpha=0.75))
                axs[i,1].text(x=right,y=bottom,s=f'{round(ssimxxhat[i].item(),2)}\n{round(psnrxxhat[i].item(),2)}',horizontalalignment='left',verticalalignment='top', bbox=dict(facecolor='white', alpha=0.75))
    
    fig.suptitle(f'Denoising of Some Samples in Different Epochs', fontsize=20, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    name = f"IMAGINE_RESULTS/VISUALISE_IMAGINE_0.8_LARGEEPOCH_b{b}{epoch_list}.png".replace(',', '_').replace("]","").replace("[","_").replace(" ","")
    plt.savefig(name)     
    plt.show()
    return ssim_psnr_values

def save_images_epochs(b, epoch_list):
    
    shared_params = {
     'batch_size': 8,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 1,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3, # L2 regularization
     'p':1
    }

    print("=======================================")
    print("Noise_Level", b, "epoch range", epoch_list)
    ssim_psnr_values = visualise_images_epochs(epoch_list, b, shared_params)
    return ssim_psnr_values

b = 0.6
epoch_list = [10,100,180]
ssim_values = save_images_epochs(b, epoch_list)



# Visualisation on The Natural Light Images

In [ ]:
def visualise_images_natural(model_file, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(shared_params["batch_size"],2 ,figsize=(25,10), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    model.cuda()
    params = { 
        "gamma": 0.01,
        "brightness": 0 # is noise level
        }
    params = {**shared_params, **params}
    experiment_modules = set_experiment(model, params)
    model.eval()
    
    
    with torch.inference_mode():
        x, noisy_x, y = next(iter(experiment_modules["natural_loader"]))
        x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
        outputs = model(x) 
        x_hat = outputs['x_hat']
        natural_labels = ["A","B","C","D","E","F","G","H","I","J"]
        #plot images
        for i in range(params["batch_size"]):
            axs[i,0].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #original image
            axs[i,1].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
            axs[i,0].set_xticklabels([])
            axs[i,0].set_yticklabels([])
            axs[i,1].set_xticklabels([])
            axs[i,1].set_yticklabels([])
            axs[i,0].set_aspect('equal')
            axs[i,1].set_aspect('equal')
            
            axs[i,0].set_ylabel(natural_labels[i], fontweight = "bold", fontsize = 25, rotation = 0)
            if i == params["batch_size"]-1:
                axs[i,0].set_xlabel("Original Image", fontweight = "bold", fontsize = 25)
                axs[i,1].set_xlabel("Denoised Image", fontweight = "bold", fontsize = 25)

    fig.suptitle(f'De-noising Naturally bad light Images', fontsize=30, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)
    name = f"IMAGINE_RESULTS/VISUALISE_imagine_0.8_Natural.png"
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_natural(model_b):
    
    shared_params = {
     'batch_size': 10,
     'random_seed': 1996,
     'image_size': (224,224),
     'n_classes': 1,
     'p':0
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"

    print("=======================================")
    print("model_file", model_file, "model_b", model_b)
    ssim_psnr_values = visualise_images_natural(model_file, shared_params)
    return ssim_psnr_values
ssim_psnr_values = save_images_natural(0.8)



In [ ]:
def visualise_images_dark(model_file, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(2, shared_params["batch_size"] ,figsize=(25,10), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    model.cuda()
    params = { 
        "gamma": 0.01,
        "brightness": 0 # is noise level
        }
    params = {**shared_params, **params}
    experiment_modules = set_experiment(model, params)
    natural_labels = ["A","B","C","D","E"]
    model.eval()
    
    
    with torch.inference_mode():
        x, noisy_x, y = next(iter(experiment_modules["dark_loader"]))
        x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
        outputs = model(x) 
        x_hat = outputs['x_hat']
    
        #plot images
        for i in range(params["batch_size"]):
            axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #original image
            #axs[i,b*2+1].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
            axs[1,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
            axs[0,i].set_xticklabels([])
            axs[0,i].set_yticklabels([])
            axs[1,i].set_xticklabels([])
            axs[1,i].set_yticklabels([])
            axs[0,i].set_aspect('equal')
            axs[1,i].set_aspect('equal')
            
            axs[1,i].set_xlabel(natural_labels[i], fontweight = "bold", fontsize = 25)
            if i == 0:
                axs[0,i].set_ylabel("Original", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=80)
                axs[1,i].set_ylabel("SDAE", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=60)

    fig.suptitle(f'De-noising Naturally Dark Images', fontsize=30, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout() # made adjust not work
    name = f"IMAGINE_RESULTS/VISUALISE_imagine_0.8_DARK.png"
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_dark(model_b):
    
    shared_params = {
     'batch_size': 5,
     'random_seed': 1996,
     'image_size': (224,224),
     'n_classes': 1,
     'p':0
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
    print("=======================================")
    print("model_file", model_file, "model_b", model_b)
    ssim_psnr_values = visualise_images_dark(model_file, shared_params)

    #with open(f'exp1_benchmark/MODEL{model_b}_SSIM_PSNR_TABLE.pickle','wb') as handle: 
    #    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ssim_psnr_values
ssim_psnr_values = save_images_dark(0.8)



In [ ]:
def visualise_images_light(model_file, shared_params):
    ssim_psnr_values = {"ssim_noisy_x":[],"ssim_xhat_x":[],"psnr_noisy_x":[],"psnr_xhat_x":[]} # b_list x B x 1 each
    fig, axs = plt.subplots(2, shared_params["batch_size"] ,figsize=(25,10), subplot_kw={'xticks': [], 'yticks': []}) #hight same as batch_size, width = GT + len(b_list) *2
    # build a rectangle in axes coords
    left, width = .25, .5
    bottom, height = .25, .5
    right = left + width
    top = bottom + height
    model = torch.load(model_file)
    model.cuda()
    params = { 
        "gamma": 0.01,
        "brightness": 0 # is noise level
        }
    params = {**shared_params, **params}
    experiment_modules = set_experiment(model, params)
    natural_labels = ["A","B","C","D","E"]
    model.eval()
    
    
    with torch.inference_mode():
        x, noisy_x, y = next(iter(experiment_modules["light_loader"]))
        x, noisy_x, y = x.cuda(), noisy_x.cuda(), y.cuda()
        outputs = model(x) 
        x_hat = outputs['x_hat']
        #plot images
        for i in range(params["batch_size"]):
            axs[0,i].imshow(np.transpose(x[i].to("cpu"), (1, 2, 0))) #original image
            #axs[i,b*2+1].imshow(np.transpose(noisy_x[i].to("cpu"), (1, 2, 0))) #noisy
            axs[1,i].imshow(np.transpose(x_hat[i].to("cpu"), (1, 2, 0)))
            axs[0,i].set_xticklabels([])
            axs[0,i].set_yticklabels([])
            axs[1,i].set_xticklabels([])
            axs[1,i].set_yticklabels([])
            axs[0,i].set_aspect('equal')
            axs[1,i].set_aspect('equal')
            
            axs[1,i].set_xlabel(natural_labels[i], fontweight = "bold", fontsize = 25)
            if i == 0:
                axs[0,i].set_ylabel("Original", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=80)
                axs[1,i].set_ylabel("SDAE", fontweight = "bold", fontsize = 25, rotation = 0, labelpad=60)

    fig.suptitle(f'De-noising Naturally Light Images', fontsize=30, fontweight="bold")
    fig.subplots_adjust(wspace=0, hspace=0)
    fig.tight_layout() # made adjust not work
    name = f"IMAGINE_RESULTS/VISUALISE_imagine_0.8_LIGHT.png"
    plt.savefig(name) 
    plt.show()
    return ssim_psnr_values


# save the psnr ssim table
def save_images_light(model_b):
    
    shared_params = {
     'batch_size': 5,
     'random_seed': 1996,
     'image_size': (224,224),
     'n_classes': 1,
     'p':0
    }
    
    model_file = f"MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
    print("=======================================")
    print("model_file", model_file, "model_b", model_b)
    ssim_psnr_values = visualise_images_light(model_file, shared_params)
    return ssim_psnr_values
ssim_psnr_values = save_images_light(0.8)



# Loading the tables to use in the report

In [ ]:
def get_table(ssim_psnr_table_path):
    with open(ssim_psnr_table_path, 'rb') as f:
        x = pickle.load(f)
    df = pd.DataFrame.from_dict(x)
    #ssim with 3 decimal
    df["mean_ssim_noisy_x"] = df["mean_ssim_noisy_x"].round(2)
    df["mean_ssim_xhat_x"] = df["mean_ssim_xhat_x"].round(2)
    # psnr with 2 decimal
    df["mean_psnr_noisy_x"] = df["mean_psnr_noisy_x"].round(2)
    df["mean_psnr_xhat_x"] = df["mean_psnr_xhat_x"].round(2)
    return df

In [ ]:
ssim_psnr_table_path = "IMAGINE_RESULTS/MODEL0.8_SSIM_PSNR_TABLE.pickle" # is with large epoch numbers
df = get_table(ssim_psnr_table_path)


In [ ]:
display(df)

# Model Analysis

In [ ]:
# https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573
def layer_visualise(model, experiment_modules):
    x, noisy_x, y = next(iter(experiment_modules["added_report_loader"]))
    outputs = []
    names = ["image", "conv1", "conv2", "conv3", "conv4", "deconv4", "deconv3", "deconv2", "deconv1"]

    outputs.append(x[3])
    image1 = model.conv1(noisy_x)
    image1 = model.act_fn(image1)
    outputs.append(image1[3])
    image2 = model.conv2(image1)
    image2 = model.act_fn(image2)
    outputs.append(image2[3])
    image3 = model.conv3(image2)
    image3 = model.act_fn(image3)
    outputs.append(image3[3])
    image4 = model.conv4(image3)
    image4 = model.act_fn(image4)
    outputs.append(image4[3])

    image5 = model.deconv4(image4)
    image5 = model.act_fn(image5)
    outputs.append(image5[3])

    image6 = model.deconv3(torch.cat((image5,image3),1))
    image6 = model.act_fn(image6)
    outputs.append(image6[3])

    image7 = model.deconv2(torch.cat((image6,image2),1))
    image7 = model.act_fn(image7)
    outputs.append(image7[3])

    image8 = model.deconv1(torch.cat((image7,image1),1))
    image8 = model.out_fn(image8)
    outputs.append(image8[3])
    #from 3d to 2D
    processed = []
    for feature_map in outputs:
        feature_map = feature_map.squeeze(0)
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0]
        processed.append(gray_scale.data.cpu().numpy())

    fig, axs = plt.subplots(2,5,figsize=(30,15))
    fig.delaxes(axs[1,4])

    for i in range(5):
        im = axs[0,i].imshow(processed[i], cmap='gray')
        divider = make_axes_locatable(axs[0,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')

        axs[0,i].axis("off")
        axs[0,i].set_title(names[i].split('(')[0], fontsize=25)
    for i in range(4):
        im = axs[1,i].imshow(processed[i+5], cmap='gray')
        divider = make_axes_locatable(axs[1,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        axs[1,i].axis("off")
        axs[1,i].set_title(names[i+5].split('(')[0], fontsize=25)
    fig.suptitle('Feature Maps of a Sample', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    name = f"IMAGINE_RESULTS/MODEL_IMAGINE_far.png"
    plt.savefig(name) 
    plt.show()
    return

In [ ]:
def get_layer_visualise(model_file):

    shared_params = {
     'num_epochs': 180,
     'batch_size': 5,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.0001, 
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 0.4 # from 0.8 til 1.2
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    layer_visualise(model, experiment_modules)
    return

In [ ]:
model_file1 = "MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
get_layer_visualise(model_file1)


# visualising the first and last feature maps In RGB

In [ ]:
# https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573
def layer_visualise(model, experiment_modules):
    x, noisy_x, y = next(iter(experiment_modules["train_loader"]))
    outputs = []
    outputs.append(x)
    outputs.append(noisy_x)
    model.eval()
    with torch.inference_mode():
        result = model(noisy_x)
        x_hat = result["x_hat"]
        outputs.append(x_hat)
        
    fig, axs = plt.subplots(3,3,figsize=(30,15))
    names = ["GT", "Noisy", "CNN-SDAE"]
    for i in range(3):
        feature_map = outputs[i]
        print(feature_map.size())
        feature_map = feature_map.squeeze(0)
        im1 = axs[0,i].imshow(feature_map[0, :, :].data.cpu().numpy(), cmap='Reds')
        im2 = axs[1,i].imshow(feature_map[1, :, :].data.cpu().numpy(), cmap='Greens')
        im3 = axs[2,i].imshow(feature_map[2, :, :].data.cpu().numpy(), cmap='Blues')

        divider = make_axes_locatable(axs[0,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im1, cax=cax, orientation='vertical')
        axs[0,i].axis("off")
        axs[0,i].set_title(names[i], fontsize=25)
        
        divider = make_axes_locatable(axs[1,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im2, cax=cax, orientation='vertical')
        axs[1,i].axis("off")
        
        divider = make_axes_locatable(axs[2,i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im3, cax=cax, orientation='vertical')
        axs[2,i].axis("off")
    fig.suptitle('The Ground Truth, Noisy, and Reconstruction of a Sample in RGB', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1)
    name = f"IMAGINE_RESULTS/MODEL_RGB_IMAGINE_0.2.png"
    plt.savefig(name) 
    plt.show()
    return

def get_layer_visualise(model_file):

    shared_params = {
     'num_epochs': 180,
     'batch_size': 1,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.0001, 
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness":0.2 # from 0.8 til 1.2
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    layer_visualise(model, experiment_modules)
    return
model_file1 = "MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
get_layer_visualise(model_file1)


# Visualising filters of the first convolution in each color

In [ ]:
def plot_filters_single_channel(t):
    #https://github.com/Niranjankumar-c/DeepLearning-PadhAI/blob/master/DeepLearning_Materials/6_VisualizationCNN_Pytorch/CNNVisualisation.ipynb
    nplots = t.shape[0]*t.shape[1]
    ncols = 12
    
    nrows = 1 + nplots//ncols
    #convert tensor to numpy image
    npimg = np.array(t.numpy(), np.float32)
    
    count = 0
    fig = plt.figure(figsize=(20,30))
    print(t.shape[0],t.shape[1])
    #looping through all the kernels in each channel
    for i in range(t.shape[0]):
        for j in range(t.shape[1]):
            count += 1
            ax1 = fig.add_subplot(nrows, ncols, count)
            npimg = np.array(t[i, j].numpy(), np.float32)
            npimg = (npimg - np.mean(npimg)) / np.std(npimg)
            npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
            if j == 0:
                ax1.imshow(npimg, cmap='Reds')
            if j == 1:
                ax1.imshow(npimg, cmap='Greens')
            if j == 2:
                ax1.imshow(npimg, cmap='Blues')
            
            ax1.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
    fig.suptitle('Filters of The Last Deconvolution', fontsize=35, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top = 0.95, bottom = 0.05)
    name = f"IMAGINE_RESULTS/dec_kernel_imagine.png"
    plt.savefig(name) 
    plt.tight_layout()
    
    plt.show()
    
def get_filters(t):
    shared_params = {
     'num_epochs': 180,
     'batch_size': 1,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.0001, 
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 0.2 # from 0.8 til 1.2
        }
    params = {**shared_params, **params} 
    experiment_modules = set_experiment(model, params)
    plot_filters_single_channel(t)
    return
model_file = "MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
model = torch.load(model_file)
model.to("cpu")
weight_tensor = model.deconv1.weight.data 
get_filters(weight_tensor)



# issue with too bright added noise on the bear 

In [ ]:
# https://ravivaishnav20.medium.com/visualizing-feature-maps-using-pytorch-12a48cd1e573

def gray_visualise(model, experiment_modules):
    x, noisy_x, y = next(iter(experiment_modules["added_report_loader"]))
    outputs0 = [x[0], noisy_x[0]]
    #from 3d to 2D
    processed = []
    for feature_map in outputs0:
        feature_map = feature_map.squeeze(0)
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0]
        processed.append(gray_scale.data.cpu().numpy())
        
    outputs1 = [x[4], noisy_x[4]]
    #from 3d to 2D
    for feature_map in outputs1:
        feature_map = feature_map.squeeze(0)    
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0] 
        processed.append(gray_scale.data.cpu().numpy())
    fig, axs = plt.subplots(2,2,figsize=(30,30))

    for i in range(2):
    
        im = axs[i,0].imshow(processed[i], cmap='gray')
        divider = make_axes_locatable(axs[i,0])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical')
        axs[i,0].axis("off")
        
        im2 = axs[i,1].imshow(processed[i+2], cmap='gray')
        divider = make_axes_locatable(axs[i,1])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im2, cax=cax, orientation='vertical')
        axs[i,1].axis("off")       
    axs[0,1].set_ylabel("GT", fontweight = "bold", fontsize = 50, rotation = 0, labelpad=30)
    axs[1,0].set_ylabel("Noisy (1.8)", fontweight = "bold", fontsize = 30, rotation = 0, labelpad=30)    
fig.suptitle('Ground Truth and The Noise Ige (Gray Scale)', fontsize=50, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95, bottom=0.05)
    name = f"IMAGINE_RESULTS/grayscale_gt_noisy.jpeg"
    plt.savefig(name) 
    plt.show()
    return

def get_gray_visualise(model_file):

    shared_params = {
     'num_epochs': 180,
     'batch_size': 7,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.0001,
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 1.8 # from 0.8 til 1.2
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    gray_visualise(model, experiment_modules)
    return

model_file1 = "MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
get_gray_visualise(model_file1)


# Histograms of one image

In [ ]:
def hist_visualise(model, experiment_modules):
    x, noisy_x, y = next(iter(experiment_modules["added_report_loader"]))
    model.eval()
    with torch.inference_mode():
        result = model(x)
        x_hat = result["x_hat"] 
        outputs = []

        outputs.append(x)
        outputs.append(noisy_x)
        outputs.append(x_hat)
    

    #from 3d to 2D
    processed = []
    for feature_map in outputs:
        feature_map = feature_map.squeeze(0)
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0]
        processed.append(gray_scale.data.cpu().numpy())
            
    fig, axs = plt.subplots(2, 3 ,figsize=(30,30))
    colors =["red", "green", "blue"]
    channels = ["Red_channel", "Green_channel", "Blue_channel"]
    image = ["GT", "Noisy X (1.8)", "SDAE"]
    for i in range(len(outputs)):
        feature_map = outputs[i]
        feature_map = feature_map.squeeze(0)
        axs[0,i].hist(feature_map[0,:, :].reshape((-1,)).data.cpu().numpy(), bins = 'auto', color = colors[0], range=[0, 1])
        axs[0,i].hist(feature_map[1,:, :].reshape((-1,)).data.cpu().numpy(), bins = 'auto', color = colors[1], range=[0, 1])
        axs[0,i].hist(feature_map[2,:, :].reshape((-1,)).data.cpu().numpy(), bins = 'auto', color = colors[2], range=[0, 1])
        axs[0,i].set_xlabel("Intensity", fontweight = "bold", fontsize = 25)
        axs[0,0].set_ylabel("Number of Pixels", fontweight = "bold", fontsize = 25)
        axs[1,0].imshow(np.transpose(x[0].to("cpu"), (1, 2, 0)))
        axs[1,1].imshow(np.transpose(noisy_x[0].to("cpu"), (1, 2, 0)))
        axs[1,2].imshow(np.transpose(x_hat[0].detach().to("cpu"), (1, 2, 0)))
        axs[1,i].set_xlabel(image[i], fontweight = "bold", fontsize = 25)
        axs[1,i].set_yticklabels([])
        axs[1,i].set_xticklabels([])
    fig.suptitle(f'Histograms of One Image', fontsize=30, fontweight="bold")
    fig.subplots_adjust(wspace=0.1, hspace=0.1, top=0.95)
    name = f"IMAGINE_RESULTS/imagine_hist.png"
    plt.savefig(name) 
    plt.show()
    return

def get_hist_visualise(model_file):

    shared_params = {
     'num_epochs': 180,
     'batch_size': 1,
     'random_seed': 1996,
     'optimizer_name': 'adamW',
     'learning_rate': 0.0001, 
     'image_size': (224,224),
     'degrees': (-10, 10),
     'translate': (0.0, 0.5),
     'scale': (0.5, 0.95),
     'dropout_p': 0.5,# dropout probability
     'negative_slope': 0.2, # negative slope for LeakyRelu
     'n_classes': 4,
     'alpha':1, #weight of ssim
     'weight_decay':1e-3 # L2 regularization
    }
    params = { 

            "gamma": 0.01,
            "p": 1,
            "brightness": 1.8 
        }
    params = {**shared_params, **params} 
    model = torch.load(model_file)
    model.to("cpu")
    experiment_modules = set_experiment(model, params)
    hist_visualise(model, experiment_modules)
    return

model_file1 = "MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8/MAIN_BENCHMARK_LRSLOW_DROPLARGE_NOISE_EPOCHNUMBERLARGE_GAMMA0.05_P0.6_BRIGHTNESS0.8epoch_180"
get_hist_visualise(model_file1)
